In [1]:
import os
os.chdir('../')

In [2]:
os.getcwd()

'/media/nazirnayal/DATA/projects/attention_long_tail/paying-attention-to-the-long-tail'

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import wandb
import pytorch_lightning as pl
import yaml
from transformer_pl import Transformer
from pytorch_lightning.loggers import WandbLogger

In [16]:
def read_config(path):
    with open(path, "r") as f:
        config = yaml.load(f, Loader=yaml.FullLoader)
    return config

config = read_config('configs/cifar10_config_local.yaml')
config

{'transformer_model': 'vit_base_patch16_224',
 'model_provider': 'timm',
 'pretrained': True,
 'num_classes': 10,
 'dataset': 'cifar10',
 'loss_function': 'cross_entropy',
 'optimizer': 'Adam',
 'learning_rate': 0.001,
 'weight_decay': 0.001,
 'class_names': ['plane',
  'car',
  'bird',
  'cat',
  'deer',
  'dog',
  'frog',
  'horse',
  'ship',
  'truck'],
 'data_root': '/media/nazirnayal/DATA/datasets/cifar10',
 'imb_type': 'exp',
 'imb_factor': 0.1,
 'random_seed': 0,
 'test_ratio': 0.8,
 'batch_size': 32,
 'num_workers': 15,
 'num_epochs': 2,
 'img_size': 224}

In [17]:
pl.seed_everything(config['random_seed'])

Global seed set to 0


0

In [18]:
model = Transformer(config)
wandb_logger = WandbLogger(name='test_run', project='paying_attention_to_the_long_tail', job_type='train', log_model='all')

trainer = pl.Trainer(
    max_epochs=config['num_epochs'],
    enable_checkpointing=True,
    fast_dev_run=False,
    overfit_batches=40,
    gpus=1,
    logger=wandb_logger,
    log_every_n_steps=1,
    profiler='simple',
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [19]:
trainer.fit(model)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
wandb: wandb version 0.12.7 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade



  | Name                | Type             | Params
---------------------------------------------------------
0 | transformer         | TimmViT          | 85.8 M
1 | loss_func           | CrossEntropyLoss | 0     
2 | acc_train           | Accuracy         | 0     
3 | acc_valid           | Accuracy         | 0     
4 | acc_test            | Accuracy         | 0     
5 | acc_per_class_train | ModuleList       | 0     
6 | acc_per_class_valid | ModuleList       | 0     
7 | acc_per_class_test  | ModuleList       | 0     
---------------------------------------------------------
85.8 M    Trainable params
0         Non-trainable params
85.8 M    Total params
343.225   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/home/nazirnayal/anaconda3/envs/nazir_env/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:432: UserWarning: You requested to overfit but enabled val/test dataloader shuffling. We are turning it off for you.
  rank_zero_warn(
Global seed set to 0
/home/nazirnayal/anaconda3/envs/nazir_env/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:591: UserWarning: You requested to overfit but enabled training dataloader shuffling. We are turning off the training dataloader shuffling for you.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

FIT Profiler Report

Action                             	|  Mean duration (s)	|Num calls      	|  Total time (s) 	|  Percentage %   	|
--------------------------------------------------------------------------------------------------------------------------------------
Total                              	|  -              	|_              	|  64.047         	|  100 %          	|
--------------------------------------------------------------------------------------------------------------------------------------
run_training_epoch                 	|  26.622         	|2              	|  53.244         	|  83.132         	|
run_training_batch                 	|  0.20142        	|80             	|  16.114         	|  25.16          	|
optimizer_step_with_closure_0      	|  0.19159        	|80             	|  15.327         	|  23.931         	|
on_train_epoch_end                 	|  6.4906         	|2              	|  12.981         	|  20.268         	|
training_step_and_backward         

In [20]:
trainer.test(model)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'class_0_test_accuracy': 1.0, 'test_accuracy': 1.0}
--------------------------------------------------------------------------------


TEST Profiler Report

Action                             	|  Mean duration (s)	|Num calls      	|  Total time (s) 	|  Percentage %   	|
---------------------------------------------------------------------------------------------------------------------------------------
Total                              	|  -              	|_              	|  113.19         	|  100 %          	|
---------------------------------------------------------------------------------------------------------------------------------------
run_training_epoch                 	|  26.622         	|2              	|  53.244         	|  47.038         	|
run_training_batch                 	|  0.20142        	|80             	|  16.114         	|  14.236         	|
optimizer_step_with_closure_0      	|  0.19159        	|80             	|  15.327         	|  13.541         	|
on_train_epoch_end                 	|  6.4906         	|2              	|  12.981         	|  11.468         	|
evaluation_step_and_end         

[{'test_accuracy': 1.0, 'class_0_test_accuracy': 1.0}]

In [14]:
wandb.finish()

In [15]:
trainer.logger.experiment.finish(exit_code=1)